# DICOM-to-BIDS conversion using `heudiconv` and `singularity`

* This notebook creates jobs for each participant and executes them by looping across participants in this notebook or by creating slurm jobs that can be run in parallel on the slurm cluser
 

#### HISTORY

* 06/14/22 aresni - refactored code to accept Config.json
* 9/9/21 dcosme - initial code

## Do the DICOM to BIDS conversion for new data using existing config setup

* Now we have set up a mapping and edited the `heuristics.py` file we can do dicom2bids conversion with the following Singularity command.



* The only thing that needs to be changed is the following:
   ```
   subs = ['']
   session = '' (If multiple sessions)
   ```
*   Edit the raw_subject_template if necessary depending on the where the raw data is and how it is named 
    For example, if raw dicoms are saved as 'sub01_raw', change '{subject}' to '{subject}_raw'
   
   Run the cell below to define everything and check the output


### Initialize variables

In [ ]:
import os

config_path = "../Config.json"
with open(config_path, 'r') as f:
    config = json.load(f)
path_object = "Environment"

project_name = config.get("Description").get("project_name")

project_directory = config.get(path_object).get( # Path to base project directory
    'project_path','/data00/projects/{project_name}').format(project_name = project_name) 

raw_directory = config.get(path_object).get(   # Path to base raw fmri directory
    'raw_path','/fmriDataRaw/fmri_data_raw') 

heudiconv_image_path = config.get(path_object).get(    # path to heudiconv image
    "heudiconv_image_path", 'Config["Environment"]["heudiconv_image_path"] Not Found')

bids_data_directory = config.get(path_object).get(    # path to where your bids_data will be stored within base
        "bids_data_directory", "data/bids_data")

bids_scripts_directory = config.get(path_object).get(       # path to where this script lives (scripts/BIDS)
        "bids_script_directory", "scripts/BIDS")

job_directory = os.path.join(
    project_directory,
    bids_scripts_directory,
    'jobs')


job_file_template = r'''#!/bin/bash
singularity run --cleanenv \
    -B {project_directory}:/base  \
    -B {raw_directory}:/raw \
    {heudiconv_image_path} \
    -d /raw/{project_name}/{raw_subject_template} \
    -o /base/{bids_data_directory} \
    -f /base/{bids_scripts_directory}/heudiconv/code/heuristic.py {session_flag}-s {pID} -c dcm2niix -b --overwrite
'''

print('Project:', project_name)
print('\nDirectories:')
print('Project Directory:', project_directory)
print('Raw Dicom Project Directory:', os.path.join(raw_directory, project_name))
print('BIDS Data Directory:', os.path.join(project_directory, bids_data_directory))
print('BIDS Scripts Directory:', os.path.join(project_directory, bids_scripts_directory))
print('Jobs directory:', os.path.join(project_directory, bids_scripts_directory, 'jobs'))
print('Heudiconv Singularity Image:', heudiconv_image_path)
print('\nExample Job: ')
print(job_file_template.format(
            project_directory = project_directory,
            raw_directory = raw_directory,
            heudiconv_image_path = heudiconv_image_path,
            project_name = project_name,
            raw_subject_template = '{subject}/*/*.dcm',
            bids_data_directory = bids_data_directory,
            bids_scripts_directory = bids_scripts_directory,
            session_flag = session_flag,
            pID = 'sub00'))

## Define Subjects and Sessions

In [ ]:
subs = ['']
session = '' # Run one session at a time. If only one session, leave as empty string

raw_subject_template = os.path.join(
    '{subject}', # Keep the '{subject}' string, but feel free to add any suffixes e.g. '{subject}_session01'
    '*',
    '*.dcm'      # Pattern matching to get all of the raw dicoms in the subject's folder  
)

## Create Jobs (and slurm)

In [ ]:
session_flag = ''
if session:
    session_flag = '-ss ' + session + ' '

if os.path.exists(job_directory) == False:
    os.mkdir(job_directory)

# loop over participants
for s in subs:
    #print('-------------- Creating: {} -------------'.format(s))
    if session:
        file_path = os.path.join(job_directory, 'heudiconv_{}_ses-{}.job').format(s,session)
    else:
        file_path = os.path.join(job_directory, 'heudiconv_{}.job').format(s)
        
    #print(file_path)
    job_file = job_file_template.format(
           project_directory = project_directory,
           raw_directory = raw_directory,
           heudiconv_image_path = heudiconv_image_path,
           project_name = project_name,
           raw_subject_template = raw_subject_template,
           bids_data_directory = bids_data_directory,
           bids_scripts_directory = bids_scripts_directory,
           session_flag = session_flag,
           pID = s)
    try:
        with open(file_path.format(s), 'w') as job:
           job.write(job_file)
        print("-------------- Job Created For: {} -------------\n".format(s), job_file)
    except IOError as e:
        print ("I/O error({0}): {1}".format(e.errno, e.strerror))
    except: #handle other exceptions such as attribute errors
        print ("Unexpected error:", sys.exc_info()[0])



### Run Jobs Within Notebook

In [ ]:
# loop over participants
for s in subs:
    print('-------------- Running: {} -------------'.format(s))
    
    if session:
        file_path = os.path.join(job_directory, 'heudiconv_{}_ses-{}.job').format(s,session)
    else:
        file_path = os.path.join(job_directory, 'heudiconv_{}.job').format(s)
        
    print(file_path)

    !bash $file_path
    print('-------------- Subject {} Done -------------'.format(s))


### Check output

In [ ]:
import glob
for s in subs:
    for filename in glob.glob(
        os.path.join(
            project_directory, 
            bids_data_directory,
            'sub-{}'.format(s),
            '*/**'
        ), recursive=True):
        print(filename)

## Create Slurm Jobs
The code below creates a job that can be run using slurm

### Define variables

In [ ]:
slurm_directory = os.path.join(job_directory, 'slurm/heudiconv')
os.makedirs(slurm_directory, exist_ok=True)

slurm_job_file_template = r'''#!/bin/bash
#SBATCH --job-name=heudiconv_{ID}.job
#SBATCH --output=out/heudiconv_{ID}.out
#SBATCH --error=out/heudiconv_{ID}.err
#SBATCH --time=02:00

srun singularity run --cleanenv \
    -B {project_directory}:/base  \
    -B {raw_directory}:/raw \
    {heudiconv_image_path} \
    -d {raw_subject_template} \
    -o {bids_data_directory} \
    -f {bids_scripts_directory}/heudiconv/code/heuristic.py {session_flag}-s {ID} -c dcm2niix -b --overwrite
'''

### Loop through specified participants

In [ ]:
for s in subs:
    print('-------------- Creating: {} -------------'.format(s))
    
    if session:
        file_path = os.path.join(slurm_directory, 'heudiconv_{}_ses-{}.job').format(s,session)
    else:
        file_path = os.path.join(slurm_directory, 'heudiconv_{}.job').format(s)
        
    print(file_path + '\n')
    
    slurm_job_file = slurm_job_file_template.format(
       project_directory = project_directory,
       raw_directory = raw_directory,
       heudiconv_image_path = heudiconv_image_path,
       raw_subject_template = raw_subject_template,
       bids_data_directory = bids_data_directory,
       bids_scripts_directory = bids_scripts_directory,
       session_flag = session_flag,
       ID = s)

    with open(file_path.format(s), 'w') as job:
        job.write(slurm_job_file)

        

### Schedule the jobs on slurm

Login to the slurm cluster:

```
ssh <JANUS_UN>@asc.upenn.edu@cls000
```

This will give you a terminal on the SLURM master node where you can look at the process queue and schedule jobs by pasting the output from the next chunk

In [ ]:
print(f"cd {slurm_directory}")
if session:
    for s in subs:
        print(f"sbatch -D {slurm_directory} -c 8 heudiconv_{s}_ses-{session}.job")
else:
    for s in subs:
        print(f"sbatch -D {slurm_directory} -c 8 heudiconv_{s}.job")
